In [4]:
import numpy as np
import tensorflow as tf
import celeriteflow as cf

In [5]:
T = tf.float64

np.random.seed(42)
N = 5000
x = tf.constant(np.sort(np.random.uniform(0, 100, N)))
diag = tf.constant(np.random.uniform(0.001, 0.01, N))
y = tf.sin(x)

log_S0 = tf.Variable(0.0, dtype=T)
log_w0 = tf.Variable(0.0, dtype=T)
log_Q = tf.Variable(0.0, dtype=T)

kernel = cf.terms.SHOTerm(log_S0=log_S0,
                          log_w0=log_w0,
                          log_Q=log_Q)

gp = cf.GaussianProcess(kernel, x, y, diag)

loglike = gp.log_likelihood
grad_loglike = tf.gradients(loglike, [log_S0, log_w0, log_Q])

In [6]:
grad_loglike

[<tf.Tensor 'gradients_1/SHOTerm_1/S0_grad/mul:0' shape=() dtype=float64>,
 <tf.Tensor 'gradients_1/SHOTerm_1/w0_grad/mul:0' shape=() dtype=float64>,
 <tf.Tensor 'gradients_1/SHOTerm_1/Q_grad/mul:0' shape=() dtype=float64>]

In [7]:
solver = cf.Solver(kernel, x, diag)
alpha = solver.apply_inverse(y[:, None])
logdet = solver.log_determinant